## Singular Value Decomposition. Save U V S into npy files.

In [1]:
import config as c
from load_model import load_model
from huggingface_utils import HuggingFaceUtils
import matplotlib.pyplot as plt
import numpy as np
import os

model_names = ["google/vit-base-patch16-224",
"google/vit-base-patch32-384",
"google/vit-large-patch16-224",
"google/vit-large-patch32-384",
"google/vit-huge-patch14-224-in21k",
"facebook/dino-vitb16",
"facebook/dino-vits16",
# "facebook/dinov2-base",
# "facebook/dinov2-small",
# "facebook/dinov2-large",
# "facebook/dinov2-giant",
"openai/clip-vit-base-patch16",
"openai/clip-vit-base-patch32",
"openai/clip-vit-large-patch14",
"facebook/deit-base-distilled-patch16-224",
"facebook/deit-small-distilled-patch16-224",
"facebook/deit-tiny-distilled-patch16-224"]

for model_name in model_names:
    (model, _) = load_model(model_name) 

    analyzer = HuggingFaceUtils(model)
    QK=analyzer.get_QK(model.config.num_attention_heads)

    U_total = [] # the vector itself
    Vt_total = []
    S_total = []

    for layer in range(model.config.num_hidden_layers):
        (Q, K, plot_path) = QK[layer]
        
        U_head = []
        Vt_head = []
        S_head = []
        for head in range(model.config.num_attention_heads):
            M = np.zeros((Q.shape[0], Q.shape[0]))
            M[:,:] = np.matmul(Q[:,head,:], K[:,head,:].T) # the interaction matrix
            U, S, Vt, angles = analyzer.svd_angles(M[:,:])
            U_head.append(U) # left singular vectors
            Vt_head.append(Vt) # right singular vectors
            S_head.append(S) # singular values
        U_total.append(U_head)
        Vt_total.append(Vt_head)
        S_total.append(S_head)
        
    embeding_size = Q.shape[2]
    U_total = np.array(U_total)[:,:,:,:embeding_size] # because M is low rank, only save up to the rank.
    Vt_total = np.array(Vt_total)[:,:,:embeding_size,:]
    S_total = np.array(S_total)[:,:,:embeding_size]

    file_path = os.path.join(c.data_path, "UVS", f"{model_name}_U_total.npy")
    if not os.path.exists(os.path.dirname(file_path)):
        os.makedirs(os.path.dirname(file_path))
    np.save(file_path, U_total)

    file_path = os.path.join(c.data_path, "UVS", f"{model_name}_Vt_total.npy")
    np.save(file_path, Vt_total)

    file_path = os.path.join(c.data_path, "UVS", f"{model_name}_S_total.npy")
    np.save(file_path, S_total)

/home/work/xupan/anaconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## plot the averageed cosine similarity between U and V.

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import os

model_names = ["google/vit-base-patch16-224",
"google/vit-base-patch32-384",
"google/vit-large-patch16-224",
"google/vit-large-patch32-384",
"google/vit-huge-patch14-224-in21k",
"facebook/dino-vitb16",
"facebook/dino-vits16",
# "facebook/dinov2-base",
# "facebook/dinov2-small",
# "facebook/dinov2-large",
# "facebook/dinov2-giant",
"openai/clip-vit-base-patch16",
"openai/clip-vit-base-patch32",
"openai/clip-vit-large-patch14",
"facebook/deit-base-distilled-patch16-224",
"facebook/deit-small-distilled-patch16-224",
"facebook/deit-tiny-distilled-patch16-224"]

f = plt.figure(figsize=(4.5,4.5),dpi=300)
for model_name, linestyle, color in [("google/vit-base-patch16-224",'-','b'),
                            ("google/vit-base-patch32-384",'--','b'),
                            ("google/vit-large-patch16-224",'-.','b'),
                            ("google/vit-large-patch32-384",':','b'),
                            ("google/vit-huge-patch14-224-in21k",'o-','b'),
                            ("facebook/dino-vits16",'-','r'),
                            ("facebook/dino-vitb16",'--','r'),
                            ("openai/clip-vit-base-patch16",'-','c'),
                            ("openai/clip-vit-base-patch32",'--','c'),
                            ("openai/clip-vit-large-patch14",'-.','c'),
                            ("facebook/deit-tiny-distilled-patch16-224",'-','m'),
                            ("facebook/deit-small-distilled-patch16-224",'--','m'),
                            ("facebook/deit-base-distilled-patch16-224",'-.','m'),
                            ]:
    U_total = np.load(os.path.join(c.data_path, "UVS", f"{model_name}_U_total.npy"))
    Vt_total = np.load(os.path.join(c.data_path, "UVS", f"{model_name}_Vt_total.npy"))
    S_total = np.load(os.path.join(c.data_path, "UVS", f"{model_name}_S_total.npy"))
    Ut_total=np.transpose(U_total, [0,1,3,2])
    cosine_similarity = np.einsum('...i,...i', Ut_total, Vt_total)
    cosine_similarity = np.mean(cosine_similarity, axis = 2)
    cosine_similarity = np.mean(cosine_similarity, axis = 1)
    plt.plot([x/(cosine_similarity.shape[0]-1) for x in range(cosine_similarity.shape[0])], cosine_similarity, color+linestyle, markersize=2, label=model_name.split('/')[1])

plt.xticks([],[])
plt.xlabel('Layer')
plt.ylabel('Cosine similarity')
plt.legend(loc='upper right', bbox_to_anchor=(0.8, 1), fontsize=6)
# plt.ylim([-1,1])

file_path=os.path.join(c.figure_path, "cosine", "average_cosine.png")
if not os.path.exists(os.path.dirname(file_path)):
    os.makedirs(os.path.dirname(file_path))
f.savefig(file_path, dpi=600, bbox_inches='tight')

plt.clf()
plt.close()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

model_names = ["google/vit-base-patch16-224",
"google/vit-base-patch32-384",
"google/vit-large-patch16-224",
"google/vit-large-patch32-384",
"google/vit-huge-patch14-224-in21k",
"facebook/dino-vitb16",
"facebook/dino-vits16",
# "facebook/dinov2-base",
# "facebook/dinov2-small",
# "facebook/dinov2-large",
# "facebook/dinov2-giant",
"openai/clip-vit-base-patch16",
"openai/clip-vit-base-patch32",
"openai/clip-vit-large-patch14",
"facebook/deit-base-distilled-patch16-224",
"facebook/deit-small-distilled-patch16-224",
"facebook/deit-tiny-distilled-patch16-224"]

f = plt.figure(figsize=(4.5,4.5),dpi=300)
for model_name, linestyle, color in [("google/vit-base-patch16-224",'-','b'),
                            ("google/vit-base-patch32-384",'--','b'),
                            ("google/vit-large-patch16-224",'-.','b'),
                            ("google/vit-large-patch32-384",':','b'),
                            ("google/vit-huge-patch14-224-in21k",'o-','b'),
                            ("facebook/dino-vits16",'-','r'),
                            ("facebook/dino-vitb16",'--','r'),
                            ("openai/clip-vit-base-patch16",'-','c'),
                            ("openai/clip-vit-base-patch32",'--','c'),
                            ("openai/clip-vit-large-patch14",'-.','c'),
                            ("facebook/deit-tiny-distilled-patch16-224",'-','m'),
                            ("facebook/deit-small-distilled-patch16-224",'--','m'),
                            ("facebook/deit-base-distilled-patch16-224",'-.','m'),
                            ]:
    U_total = np.load(os.path.join(c.data_path, "UVS", f"{model_name}_U_total.npy"))
    Vt_total = np.load(os.path.join(c.data_path, "UVS", f"{model_name}_Vt_total.npy"))
    S_total = np.load(os.path.join(c.data_path, "UVS", f"{model_name}_S_total.npy"))
    Ut_total=np.transpose(U_total, [0,1,3,2])
    cosine_similarity = np.einsum('...i,...i', Ut_total, Vt_total)

    S_total = S_total/np.sum(S_total, axis=2, keepdims=True) # weigh cosine similarity by singulr values.
    cosine_similarity = np.multiply(cosine_similarity,S_total)

    cosine_similarity = np.mean(cosine_similarity, axis = 2)
    cosine_similarity = np.mean(cosine_similarity, axis = 1)
    plt.plot([x/(cosine_similarity.shape[0]-1) for x in range(cosine_similarity.shape[0])], cosine_similarity, color+linestyle, markersize=2, label=model_name.split('/')[1])

plt.xticks([],[])
plt.xlabel('Layer')
plt.ylabel(''Weighted cosine similarity')
plt.legend(loc='upper right', bbox_to_anchor=(0.8, 1), fontsize=6)

file_path=os.path.join(c.figure_path, "cosine", "average_weighted_cosine.png")
if not os.path.exists(os.path.dirname(file_path)):
    os.makedirs(os.path.dirname(file_path))
f.savefig(file_path, dpi=600, bbox_inches='tight')

plt.clf()
plt.close()

## Plot the spectrum and cosine similarity for each layer each head.

In [1]:
import config as c
import matplotlib.pyplot as plt
import numpy as np
import os

model_names = ["google/vit-base-patch16-224",
"google/vit-base-patch32-384",
"google/vit-large-patch16-224",
"google/vit-large-patch32-384",
"google/vit-huge-patch14-224-in21k",
"facebook/dino-vitb16",
"facebook/dino-vits16",
# "facebook/dinov2-base",
# "facebook/dinov2-small",
# "facebook/dinov2-large",
# "facebook/dinov2-giant",
"openai/clip-vit-base-patch16",
"openai/clip-vit-base-patch32",
"openai/clip-vit-large-patch14",
"facebook/deit-base-distilled-patch16-224",
"facebook/deit-small-distilled-patch16-224",
"facebook/deit-tiny-distilled-patch16-224"]

for model_name in model_names:
    print(model_name)

    U_total = np.load(os.path.join(c.data_path, "UVS", f"{model_name}_U_total.npy"))
    Vt_total = np.load(os.path.join(c.data_path, "UVS", f"{model_name}_Vt_total.npy"))
    S_total = np.load(os.path.join(c.data_path, "UVS", f"{model_name}_S_total.npy"))

    Ut_total=np.transpose(U_total, [0,1,3,2])
    cosine_similarity = np.einsum('...i,...i', Ut_total, Vt_total)

    dim = Ut_total.shape[3]
    a1 = np.random.rand(10000,dim)
    a2 = np.random.rand(10000,dim)
    a1 = a1 / np.linalg.norm(a1, axis=1, keepdims=True)
    a2 = a1 / np.linalg.norm(a2, axis=1, keepdims=True)
    cosine_similarity_control = np.einsum('...i,...i', a1, a2)
    baseline = np.percentile(cosine_similarity_control, 99)

    n_layer = cosine_similarity.shape[0]
    n_head = cosine_similarity.shape[1]

    f, axs = plt.subplots(n_layer, n_head, figsize=(n_head*4, n_layer*3),dpi=100)
    f.tight_layout()
    for layer in range(n_layer):
        sigular_value_max = np.max(S_total[layer,:,:])
        for head in range(n_head):
            ax1 = axs[layer][head]
            ax1.plot(cosine_similarity[layer,head,:], ".", color='firebrick')
            ax1.set_ylim([-1,1])
            ax1.set_xlabel('Mode')
            ax1.set_ylabel('Cosine similarity')
            ax1.axhline(y = baseline, color = 'gray', linestyle = '--') 
            ax1.axhline(y = -baseline, color = 'gray', linestyle = '--') 

            ax2 = ax1.twinx()
            ax2.set_ylim([0,1.1*sigular_value_max])
            ax2.plot(S_total[layer,head,:], "x", color='royalblue')
            ax2.set_ylabel('Singular value')
            ax2.set_title(f"L{layer}, h{head}")

            f.tight_layout()

    file_path=os.path.join(c.figure_path, "cosine", f"{model_name}_cosine_spectrum.png")
    if not os.path.exists(os.path.dirname(file_path)):
        os.makedirs(os.path.dirname(file_path))
    f.savefig(file_path, dpi=100, bbox_inches='tight')
    
    plt.clf()
    plt.close()

facebook/dinov2-small
